### Setup Colab

In [ ]:
%load_ext autoreload
%autoreload 2
#colab config
from google.colab import drive

drive.mount('/content/drive')

import os

GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'NYU ML'

GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)

os.chdir(GOOGLE_DRIVE_PATH)

print("Current working directory:", os.getcwd())

Mounted at /content/drive
Current working directory: /content/drive/My Drive/NYU ML


In [ ]:
!pip install gcsfs==2024.9.0.post1 fsspec==2024.9.0
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: gcsfs
    Found existing installation: gcsfs 2024.10.0
    Uninstalling gcsfs-2024.10.0:
      Successfully uninstalled gcsfs-2024.10.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 44.8 MB/s eta 0:00:00


# Import

In [11]:
from datasets import load_dataset
import numpy as np
import math

### Load Data

In [ ]:
total_dataset = load_dataset("MissTiny/WikiArt",cache_dir="data")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/2.21k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/66 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/66 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/22 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/60 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
datatotal_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'artist', 'date', 'genre', 'style', 'description', 'filename', 'image', 'embeddings_pca512', 'image_numpy', 'CLIPVisionModelWithProjection_image_embeds'],
        num_rows: 57529
    })
    test: Dataset({
        features: ['title', 'artist', 'date', 'genre', 'style', 'description', 'filename', 'image', 'embeddings_pca512', 'image_numpy', 'CLIPVisionModelWithProjection_image_embeds'],
        num_rows: 19177
    })
})

In [22]:
X_train = total_dataset['train']['embeddings_pca512']
y_train = total_dataset['train']['date']

X_test = total_dataset['test']['embeddings_pca512']
y_test = total_dataset['test']['date']

In [45]:
# mini dataset to overfit
X_mini = X_train[:50000]
y_mini = y_train[:0000]

def date_to_1hot(y, interval = 10):
  y_int = np.array([int(year) for year in y])
  min_year = min(y_int)
  max_year = max(y_int)
  range_year = max_year - min_year
  num_item = y_int.shape[0]
  num_features = math.ceil(range_year / interval)

  y_label = np.zeros((num_item, num_features))
  for i in range(num_item):
    index = (y_int[i] - min_year) // interval
    y_label[i][index] = 1

  return y_label

y_mini_label = date_to_1hot(y_mini)
# print(len(y_mini_label[0]), y_mini_label[0], y_mini)

In [74]:
def date_to_class(y, interval = 10):
  y_int = np.array([int(year) for year in y])
  min_year = min(y_int)
  max_year = max(y_int)
  range_year = max_year - min_year
  num_item = y_int.shape[0]
  num_features = math.ceil(range_year / interval)

  y_class = np.zeros(num_item, dtype = int)
  for i in range(num_item):
    index = (y_int[i] - min_year) // interval
    y_class[i] = min_year + index*interval

  return y_class

In [23]:
y_train_label = date_to_1hot(y_train)
y_test_label = date_to_1hot(y_test)

## Random forest classifier


Overfit Mini dataset

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [25]:
# Assuming X_train, y_train are your training data and X_test, y_test are your test data
# RandomForest classifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
classifier.fit(X_mini, y_mini_label)

# Make predictions
y_mini_pred_rf = classifier.predict(X_mini)

In [26]:
# Evaluate accuracy
accuracy = accuracy_score(y_mini_label, y_mini_pred_rf)
print(f'Accuracy: {accuracy}')

Accuracy: 1.0


The whole dataset

In [27]:
# Assuming X_train, y_train are your training data and X_test, y_test are your test data
# RandomForest classifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
classifier.fit(X_train, y_train_label)

# Make predictions
y_pred_rf = classifier.predict(X_test)

In [28]:
accuracy_rf = accuracy_score(y_test_label, y_pred_rf)
print(f'Accuracy: {accuracy_rf}')

ValueError: inconsistent shapes

# XGBoost

In [29]:
!pip install xgboost


In [71]:
y_train_c = date_to_class(y_train)
y_test_int = date_to_class(y_test)
print(y_train_c)

[85 84 87 ... 24 86 78]


In [73]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Define the XGBoost model for multi-class classification
model = xgb.XGBClassifier(
    objective='multi:softmax',  # Specify multi-class classification
    num_class=100,               # Set the number of classes
    eval_metric='mlogloss'      # Log loss metric for multi-class
)

# Train the model
model.fit(X_train, y_train_c)

# Make predictions
y_train_c_XG = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test_c, y_train_c_XG)
print(f'Accuracy: {accuracy}')

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79], got [ 0  1  2  4  5 10 12 13 14 21 22 23 24 25 26 27 28 31 34 35 36 37 38 39
 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63
 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87
 88 89 90 91 92 93 94 95]